In [1]:
import tensorflow as tf
import numpy as np
import os
import PIL
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
# Mounted google drive at session storage

#from google.colab import drive
#drive.mount('/content/drive/', force_remount = True)

In [4]:
import pathlib
import zipfile

#Modify next line according to your drive location or local os path

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Classroom/Artificial Intelligence-07B1/Cats vs Dogs Image Classification/train_ds.zip", 'r')
zip_ref.extractall("/content/data/train")  # extracted data from zip file 'train_ds.zip'

data_dir = pathlib.Path("/content/data/train/train")  # store path of train dataset
zip_ref.close()

In [ ]:
data_dir

PosixPath('/content/data/train/train')

In [ ]:
# total image count

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

25000


In [ ]:
batch_size = 32

# resize all images to size (180,180)
img_height = 180
img_width = 180

# Create training set from images directory using keras.preprocessing

train_ds = keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [ ]:
# Create validation set from images directory (train - 0.8, validation - 0.2)

val_ds = keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
classes = train_ds.class_names
print(classes)

['cat', 'dog']


In [ ]:
for images, labels in train_ds:
  images_shape = images.shape
  labels_shape = labels.shape
  print(images.shape)
  print(labels.shape)
  print(type(images))
  break

(32, 180, 180, 3)
(32,)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [ ]:
for images, labels in val_ds:
  print(images.shape)
  print(labels.shape)
  break


(32, 180, 180, 3)
(32,)


In [ ]:
from keras import Sequential
# from keras.preprocessing.image import ImageDataGenerator
# try to use ImageDataGenerator (Data augmentation e.g random flips, crops..) to improve accuracy


In [ ]:
# Define the model architecture
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model = Sequential()

model.add(keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(180, 180, 3))) # Normalize the dataset to improve accuracy

model.add(Conv2D(32, (3, 3), activation='relu'))  
model.add(BatchNormalization())  # normalize the activations after each layer
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))  # random dropping of units to prevent overfitting and improve validation accuracy

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # sigmoid activation used as only 2 classes are present

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_5 (Rescaling)      (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 178, 178, 32)      896       
_________________________________________________________________
batch_normalization_16 (Batc (None, 178, 178, 32)      128       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 89, 89, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 87, 87, 64)        18496     
_________________________________________________________________
batch_normalization_17 (Batc (None, 87, 87, 64)       

In [ ]:
model.compile(loss = keras.losses.BinaryCrossentropy(), optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# callbacks used to prevent model from diverging and reduce learning rate (by factor of 0.5) as we approach minima
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlystop = EarlyStopping(patience=10)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

In [ ]:
# Early stoppage as validation loss has become saturated
history = model.fit(train_ds, validation_data = val_ds, validation_steps = 157, steps_per_epoch = 625, epochs = 20, callbacks = callbacks)

Epoch 1/20
625/625 [==============================] - 71s 113ms/step - loss: 0.6703 - accuracy: 0.6688 - val_loss: 0.5757 - val_accuracy: 0.6992
Epoch 2/20
625/625 [==============================] - 70s 113ms/step - loss: 0.4874 - accuracy: 0.7645 - val_loss: 0.5454 - val_accuracy: 0.7370
Epoch 3/20
625/625 [==============================] - 71s 114ms/step - loss: 0.4514 - accuracy: 0.7864 - val_loss: 0.4466 - val_accuracy: 0.7904
Epoch 4/20
625/625 [==============================] - 71s 113ms/step - loss: 0.3765 - accuracy: 0.8317 - val_loss: 0.4643 - val_accuracy: 0.7774
Epoch 5/20
625/625 [==============================] - ETA: 0s - loss: 0.3209 - accuracy: 0.8605
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
625/625 [==============================] - 71s 113ms/step - loss: 0.3209 - accuracy: 0.8605 - val_loss: 0.8630 - val_accuracy: 0.6998
Epoch 6/20
625/625 [==============================] - 71s 113ms/step - loss: 0.2306 - accuracy: 0.9064 - val_l

In [ ]:
#serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# The model saved here can be used in the actual .py file to test images
# This model should only be used to train the model.

# References
# https://www.kaggle.com/uysimty/keras-cnn-dog-or-cat-classification